<a href="https://colab.research.google.com/github/KatiaKudinova/Portfolio-Project-2-A-B-Testing-Analysis/blob/main/Portfolio_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Portfolio Project 2.
Підключаємось до BigQuery.

In [ ]:
!pip install --upgrade google-cloud-bigquery

from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
client = bigquery.Client(project="data-analytics-mate")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.9/258.9 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.36.0
    Uninstalling google-cloud-bigquery-3.36.0:
      Successfully uninstalled google-cloud-bigquery-3.36.0


Робимо SQL запит до таблиці.

In [ ]:
query = """
with session_info as (
SELECT
s.date,
s.ga_session_id,
sp.country,
sp.device,
sp.continent,
sp.channel,
ab.test,
ab.test_group
from data-analytics-mate.DA.ab_test as ab
join `DA.session` as s
on ab.ga_session_id = s.ga_session_id
join `DA.session_params` as sp
on sp.ga_session_id = s.ga_session_id
),
session_with_orders as (
SELECT
session_info.date,
-- session_info.ga_session_id,
session_info.country,
session_info.device,
session_info.continent,
session_info.channel,
session_info.test,
session_info.test_group,
count(distinct o.ga_session_id) as session_with_orders
FROM `DA.order` as o
join session_info
on o.ga_session_id = session_info.ga_session_id
group by
session_info.date,
session_info.country,
session_info.device,
session_info.continent,
session_info.channel,
session_info.test,
session_info.test_group
),
event as (
SELECT
    session_info.date,
    -- session_info.ga_session_id,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    sp.event_name,
    count(sp.ga_session_id) as event_cnt
FROM `DA.event_params` as sp
join session_info
on sp.ga_session_id = session_info.ga_session_id
group by
    session_info.date,
    -- session_info.ga_session_id,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    sp.event_name
),
session as (
select
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    count(distinct session_info.ga_session_id) as session_cnt
from session_info
group by
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group
),
account as (
select
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    count(distinct acs.ga_session_id) as new_account_cnt
from `DA.account_session` as acs
join session_info
on acs.ga_session_id = session_info.ga_session_id
group by
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group
)
select
    session_with_orders.date,
    session_with_orders.country,
    session_with_orders.device,
    session_with_orders.continent,
    session_with_orders.channel,
    session_with_orders.test,
    session_with_orders.test_group,
    'session_with_orders' as event_name,
    session_with_orders.session_with_orders as value
from
session_with_orders
union all
select
    event.date,
    event.country,
    event.device,
    event.continent,
    event.channel,
    event.test,
    event.test_group,
    event_name,
    event.event_cnt as value
from event
union all
select
    session.date,
    session.country,
    session.device,
    session.continent,
    session.channel,
    session.test,
    session.test_group,
    'session' as event_name,
    session.session_cnt as value
from session
union all
select
    account.date,
    account.country,
    account.device,
    account.continent,
    account.channel,
    account.test,
    account.test_group,
    'new account' as event_name,
    account.new_account_cnt as value
from account
"""
df = client.query(query).to_dataframe()
print(df.head())

         date  country   device continent         channel  test  test_group  \
0  2020-11-02  Ecuador   mobile  Americas  Organic Search     2           2   
1  2020-11-02   Kuwait  desktop      Asia  Organic Search     2           2   
2  2020-11-06   Cyprus  desktop      Asia          Direct     2           1   
3  2020-11-06  Ecuador  desktop  Americas     Paid Search     2           1   
4  2020-11-07  Tunisia  desktop    Africa          Direct     2           1   

            event_name  value  
0  session_with_orders      1  
1  session_with_orders      1  
2  session_with_orders      1  
3  session_with_orders      1  
4  session_with_orders      1  


Перевіряємо тип отриманих даних та нульові значення.

In [ ]:
print(df.info())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800996 entries, 0 to 800995
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        800996 non-null  dbdate
 1   country     800996 non-null  object
 2   device      800996 non-null  object
 3   continent   800996 non-null  object
 4   channel     800996 non-null  object
 5   test        800996 non-null  Int64 
 6   test_group  800996 non-null  Int64 
 7   event_name  800996 non-null  object
 8   value       800996 non-null  Int64 
dtypes: Int64(3), dbdate(1), object(5)
memory usage: 57.3+ MB
None
date          0
country       0
device        0
continent     0
channel       0
test          0
test_group    0
event_name    0
value         0
dtype: int64


Перевіряємо і рахуємо події у колонці event_name.

In [ ]:
event_counts = df['event_name'].value_counts()
print(event_counts)

event_name
session                107210
session_start          106242
page_view              101907
user_engagement         94520
first_visit             81621
scroll                  73643
view_promotion          61695
view_item               44869
session_with_orders     25892
new account             22389
view_search_results     14310
add_shipping_info       11961
begin_checkout          11959
select_item             11816
add_to_cart             11762
select_promotion         9044
add_payment_info         8048
click                    1997
view_item_list            111
Name: count, dtype: int64


Імпортуємо необхідні бібліотеки, створюємо словник з метриками та зведену таблицю з total результатами.

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.proportion import proportions_ztest
import numpy as np
metrics = [{"name": "begin_checkout/session", "num": "begin_checkout", "den": "session"},
           {"name": "add_payment_info/session", "num": "add_payment_info", "den": "session"},
           {"name": "add_shipping_info/session", "num": "add_shipping_info", "den": "session"},
           {"name": "new account/session", "num": "new account", "den": "session"}]
results = []
def make_pivot(df):
        agg = df.groupby(["test", "test_group", "event_name"])["value"].sum().reset_index()

        pivot = agg.pivot_table(index=["test", "test_group"],
        columns="event_name", values="value", fill_value=0).reset_index()

        pivot.columns.name = None
        return pivot
pivot_total = make_pivot(df)
print(pivot_total)

   test  test_group  add_payment_info  add_shipping_info  add_to_cart  \
0     1           1            1988.0             3034.0       1395.0   
1     1           2            2229.0             3221.0       1366.0   
2     2           1            2344.0             3480.0       2811.0   
3     2           2            2409.0             3510.0       3061.0   
4     3           1            3623.0             5298.0      17674.0   
5     3           2            3697.0             5188.0      17229.0   
6     4           1            3731.0             5128.0      21536.0   
7     4           2            3601.0             4956.0      21554.0   

   begin_checkout  click  first_visit  new account  page_view  ...  \
0          3784.0  368.0      30596.0       3823.0   191543.0  ...   
1          4021.0  353.0      30512.0       3681.0   198050.0  ...   
2          4262.0  337.0      34511.0       4165.0   220275.0  ...   
3          4313.0  413.0      34171.0       4184.0   212320.0 

Тест пропорцій для загальних результатів.

In [ ]:
tests = pivot_total['test'].unique()

for test_id in tests:
    df_test = pivot_total[pivot_total['test'] == test_id]
    group1 =  df_test[df_test['test_group'] == 1]
    group2 =  df_test[df_test['test_group'] == 2]

    for metric in metrics:
        num1 = group1[metric['num']].sum()
        num2 = group2[metric['num']].sum()
        den1 = group1[metric['den']].sum()
        den2 = group2[metric['den']].sum()

        conv1 = num1 / den1 if den1 > 0 else 0
        conv2 = num2 / den2 if den2 > 0 else 0

        count = np.array([num1, num2])
        nobs = np.array([den1, den2])
        z_stat, p_value = proportions_ztest(count, nobs)

        metric_change = conv2 / conv1 if conv1 > 0 else np.nan
        significant = p_value < 0.05
        print(f"Metric {metric['name']}, p-value = {p_value:.4f}, z_stat = {z_stat:.4f}")
        results.append({"test_number": test_id, "metrics": metric['name'], "numerator_ev": num1, "denominator_ev": den1,
                        "numerator_cohort": num2, "denominator_cohort": den2, "conversion_rate_1": conv1, "conversion_rate_2": conv2,
                        "metric_change": metric_change, "z_stat": z_stat, "p_value": p_value, "significant": significant})

Metric begin_checkout/session, p-value = 0.0029, z_stat = -2.9788
Metric add_payment_info/session, p-value = 0.0001, z_stat = -3.9249
Metric add_shipping_info/session, p-value = 0.0092, z_stat = -2.6036
Metric new account/session, p-value = 0.1229, z_stat = 1.5429
Metric begin_checkout/session, p-value = 0.3406, z_stat = -0.9529
Metric add_payment_info/session, p-value = 0.2146, z_stat = -1.2410
Metric add_shipping_info/session, p-value = 0.4780, z_stat = -0.7096
Metric new account/session, p-value = 0.5560, z_stat = -0.5888
Metric begin_checkout/session, p-value = 0.0120, z_stat = 2.5114
Metric add_payment_info/session, p-value = 0.5201, z_stat = -0.6432
Metric add_shipping_info/session, p-value = 0.1574, z_stat = 1.4137
Metric new account/session, p-value = 0.5199, z_stat = 0.6435
Metric begin_checkout/session, p-value = 0.0459, z_stat = 1.9960
Metric add_payment_info/session, p-value = 0.1162, z_stat = 1.5711
Metric add_shipping_info/session, p-value = 0.0741, z_stat = 1.7858
Metric

In [ ]:
results_df = pd.DataFrame(results)

from google.colab import drive
drive.mount('/content/drive')

results_df.to_csv("/content/drive/MyDrive/DA/project2.csv", index=False)
results_df.to_csv("project2.csv", index=False)


Mounted at /content/drive


Посилання на [документ](https://drive.google.com/file/d/1OMaI5D2_34mY47yfHk0XvQ5_MfaYSoMZ/view?usp=sharing)

Посилання на [дашборд](https://public.tableau.com/app/profile/kateryna.kudinova/viz/ABtest_17568047571490/ABtest)

Створюємо зведену таблицю по континентам.

In [ ]:
def make_pivot_continent(df):
    agg = df.groupby(["test", "test_group", "continent", "event_name"])["value"].sum().reset_index()

    pivot = agg.pivot_table(index=["test", "test_group", "continent"], columns="event_name", values="value", fill_value=0).reset_index()
    pivot.columns.name = None
    return pivot
pivot_continent = make_pivot_continent(df)
print(pivot_continent)

    test  test_group  continent  add_payment_info  add_shipping_info  \
0      1           1  (not set)               7.0                4.0   
1      1           1     Africa              19.0               41.0   
2      1           1   Americas            1107.0             1704.0   
3      1           1       Asia             512.0              730.0   
4      1           1     Europe             324.0              528.0   
5      1           1    Oceania              19.0               27.0   
6      1           2  (not set)              10.0               13.0   
7      1           2     Africa              24.0               28.0   
8      1           2   Americas            1177.0             1738.0   
9      1           2       Asia             545.0              798.0   
10     1           2     Europe             452.0              606.0   
11     1           2    Oceania              21.0               38.0   
12     2           1  (not set)              11.0               

Проводимо тест пропорцій в розрізі континентів.

In [ ]:
tests1 = pivot_continent['test'].unique()

for test_id in tests1:
    df_test1 = pivot_continent[pivot_continent['test'] == test_id]
    group1 =  df_test1[df_test1['test_group'] == 1]
    group2 =  df_test1[df_test1['test_group'] == 2]

    for metric in metrics:
        num1 = group1[metric['num']].sum()
        num2 = group2[metric['num']].sum()
        den1 = group1[metric['den']].sum()
        den2 = group2[metric['den']].sum()

        conv1 = num1 / den1 if den1 > 0 else 0
        conv2 = num2 / den2 if den2 > 0 else 0

        count = np.array([num1, num2])
        nobs = np.array([den1, den2])
        z_stat, p_value = proportions_ztest(count, nobs)

        metric_change = conv2 / conv1 if conv1 > 0 else np.nan
        significant = p_value < 0.05
        print(f"Metric {metric['name']}, p-value = {p_value:.4f}, z_stat = {z_stat:.4f}")

Metric begin_checkout/session, p-value = 0.0029, z_stat = -2.9788
Metric add_payment_info/session, p-value = 0.0001, z_stat = -3.9249
Metric add_shipping_info/session, p-value = 0.0092, z_stat = -2.6036
Metric new account/session, p-value = 0.1229, z_stat = 1.5429
Metric begin_checkout/session, p-value = 0.3406, z_stat = -0.9529
Metric add_payment_info/session, p-value = 0.2146, z_stat = -1.2410
Metric add_shipping_info/session, p-value = 0.4780, z_stat = -0.7096
Metric new account/session, p-value = 0.5560, z_stat = -0.5888
Metric begin_checkout/session, p-value = 0.0120, z_stat = 2.5114
Metric add_payment_info/session, p-value = 0.5201, z_stat = -0.6432
Metric add_shipping_info/session, p-value = 0.1574, z_stat = 1.4137
Metric new account/session, p-value = 0.5199, z_stat = 0.6435
Metric begin_checkout/session, p-value = 0.0459, z_stat = 1.9960
Metric add_payment_info/session, p-value = 0.1162, z_stat = 1.5711
Metric add_shipping_info/session, p-value = 0.0741, z_stat = 1.7858
Metric